### R06725035 陳廷易
* t_index ,term ,df => dictionary.txt
* terms# \n t_index, tf-idf (log10) => docID.txt
* cos-similarity function between two doc => def cosine(DOCx,DOCy)

In [1]:
# import keras
# from keras.preprocessing.text import Tokenizer
# import gensim
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from nltk import word_tokenize
from nltk.stem.porter import *
from nltk.tokenize import RegexpTokenizer
import os
import re
import pickle
import pandas as pd
import numpy as np
import tqdm
from tqdm import tqdm
from multiprocessing import Pool, Manager
# nltk.download('all')

stop words and stemming prepare:

In [73]:
with open('data/stop_words.txt') as f:
    stop_words_list = f.read().splitlines() #stop_list1
stop_list2 = pickle.load(open('data/stop_list2.pkl','rb'))
ps = PorterStemmer() # Stemming
stop_words = set(stopwords.words('english')) #Stopword
short = ['.', ',', '"', "\'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}', "'at",
         "_","`","\'\'","--","``",".,","//",":","___",'_the','-',"'em",".com",
                   '\'s','\'m','\'re','\'ll','\'d','n\'t','shan\'t',"...","\'ve",'u']
stop_words_list.extend(short)
stop_words_list.extend(stop_list2)
stop_words.update(stop_words_list) # remove it if you need punctuation



# tokens = nltk.word_tokenize(texts.lower())
# ps.stem(token_result)
# token_result

In [74]:
def preprocess(texts):
    tokens = [i for i in word_tokenize(texts.lower()) if i not in stop_words]  # Tokenization.# Lowercasing
    token_result = ''
    token_result_ = ''
    for i,token in enumerate(tokens): #list2str
        token_result += ps.stem(token) + ' '
    token_result = ''.join([i for i in token_result if not i.isdigit()])
    token_result = [i for i in word_tokenize(token_result) if i not in stop_words]
    for i,token in enumerate(token_result):
        token_result_ += token + ' '
    return token_result_

get all documents cut out tokens

In [75]:
tokens_all = ""
for file in next(os.walk('data/IRTM/'))[2]:
    f = open('data/IRTM/'+file)
    texts = f.read()
    f.close()
    tokens_all += preprocess(texts)

remove too short and print

In [76]:
token_set = set(tokens_all.split(' '))
token_list = list(token_set)
for token in token_list: #13551=>13406
    if len(token)<3:
        token_list.remove(token)
token_list = sorted(token_list)
print(len(token_list))
token_list

13191


['abahd',
 'abandon',
 'abat',
 'abc',
 'abc.com',
 'abcnews.com',
 'abdallah',
 'abdel',
 'abdomin',
 'abduct',
 'abdul',
 'abdul-karim',
 'abdullah',
 'abdurahman',
 'aberr',
 'abhad',
 'abhiyan',
 'abhorr',
 'abid',
 'abidjan',
 'abijan',
 'abil',
 'abl',
 'ablaz',
 'abnorm',
 'aboard',
 'abobo',
 'abolish',
 'abomin',
 'abort',
 'abou',
 'abound',
 'abovo',
 'abraham',
 'abreast',
 'abrego',
 'abroad',
 'abrog',
 'abrupt',
 'abruptli',
 'abscam',
 'absenc',
 'absent',
 'absente',
 'absentia',
 'absolut',
 'absorb',
 'abstain',
 'abstent',
 'absurd',
 'abu',
 'abund',
 'abus',
 'abuse-of-pow',
 'abuzaakouk',
 'abyan',
 'abyssinian',
 'academ',
 'academi',
 'academia',
 'acced',
 'acceler',
 'accent',
 'accentu',
 'accept',
 'access',
 'accid',
 'accident',
 'accommod',
 'accompani',
 'accomplic',
 'accomplish',
 'accord',
 'accordingli',
 'accordion',
 'account',
 'accredit',
 'accumul',
 'accur',
 'accuraci',
 'accus',
 'accustom',
 'ace',
 'acerb',
 'achiev',
 'acid',
 'acknowledg

In [134]:
df = pd.DataFrame(pd.Series(token_list),columns=['term'])
df['t_index'] = df.index+1
df['df'] = 0
df = df[['t_index','term','df']]
df

,t_index,term,df
0,1,abahd,0
1,2,abandon,0
2,3,abat,0
3,4,abc,0
4,5,abc.com,0
5,6,abcnews.com,0
6,7,abdallah,0
7,8,abdel,0
8,9,abdomin,0
9,10,abduct,0


In [137]:
for file in tqdm(next(os.walk('data/IRTM/'))[2]):
    f = open('data/IRTM/'+file)
    texts = f.read()
    f.close()
    tokens_all = preprocess(texts)
    for term in token_list:
        if term in tokens_all:
            df.loc[df['term']==term,'df'] += 1 # build the dict
#     print(df.loc[df['term']==term,'df'])
df

100%|██████████| 1095/1095 [14:44<00:00,  1.24it/s]


,t_index,term,df
0,1,abahd,1
1,2,abandon,39
2,3,abat,1
3,4,abc,51
4,5,abc.com,1
5,6,abcnews.com,3
6,7,abdallah,2
7,8,abdel,3
8,9,abdomin,2
9,10,abduct,2


差一步:可以把一些字詞合併

In [200]:
df2 = df.drop(df[df.df<2].index)
df2 = df2.drop(df2[df2.df>950].index)
df2.reset_index(drop=True,inplace=True)
df2['t_index'] = df2.index + 1
df2  # 最後的terms dict

,t_index,term,df
0,1,abandon,39
1,2,abc,51
2,3,abcnews.com,3
3,4,abdallah,2
4,5,abdel,3
5,6,abdomin,2
6,7,abduct,2
7,8,abdul,53
8,9,abdul-karim,11
9,10,abdullah,40


dictionary output

In [210]:
df2.to_csv('output/dictionary.txt',index=False,header=False,sep=' ')
# tfile = open('output/dictionary2.txt', 'w')
# kk = df2.to_string(index=False, header=False)
# tfile.writelines(kk)
# tfile.close()
# pickle.dump(df2,open('data/dictionar_df.pkl','wb'))

count tf-idf

In [233]:
tf_list = list(df2.term)
for file in tqdm(next(os.walk('data/IRTM/'))[2]):
    df_tf = df2[['t_index','term']]
    df_tf['tf'] = 0
#     df_tf
    f = open('data/IRTM/'+file)
    texts = f.read()
    f.close()
    tokens_all = preprocess(texts)
    for term in tf_list:
        if term in tokens_all:
            df_tf.loc[df_tf['term']==term,'tf'] += 1
    df_tf.tf = (df_tf.tf / 8912)* np.log10(1095 / df2.df)
    df_tf = df_tf[df_tf.tf > 0]
    df_tf.drop('term',axis=1,inplace=True)
    df_tf.to_csv('output/tf-idf/'+file, index=False, header=False, sep=' ')
    with open('output/tf-idf/'+file,'r') as ori: data = ori.read()
    with open('output/tf-idf/'+file, 'w') as mod: mod.write(str(len(df_tf))+'\n'+data)
    
df_tf




  0%|          | 0/1095 [00:00<?, ?it/s]


  0%|          | 1/1095 [00:00<03:53,  4.68it/s]


  0%|          | 2/1095 [00:01<09:51,  1.85it/s]


  0%|          | 3/1095 [00:01<11:06,  1.64it/s]


  0%|          | 4/1095 [00:02<09:19,  1.95it/s]


  0%|          | 5/1095 [00:02<08:25,  2.16it/s]


  1%|          | 6/1095 [00:03<11:06,  1.63it/s]


  1%|          | 7/1095 [00:03<10:15,  1.77it/s]


  1%|          | 8/1095 [00:04<09:29,  1.91it/s]


  1%|          | 9/1095 [00:04<08:55,  2.03it/s]


  1%|          | 10/1095 [00:04<08:33,  2.11it/s]


  1%|          | 11/1095 [00:05<09:46,  1.85it/s]


  1%|          | 12/1095 [00:07<11:02,  1.64it/s]


  1%|          | 13/1095 [00:07<11:01,  1.64it/s]


  1%|▏         | 14/1095 [00:09<12:03,  1.49it/s]


  1%|▏         | 15/1095 [00:09<11:59,  1.50it/s]


  1%|▏         | 16/1095 [00:10<12:19,  1.46it/s]


  2%|▏         | 17/1095 [00:11<12:23,  1.45it/s]


  2%|▏         | 18/1095 [00:12<12:42,  1.41it/s]


  2%|▏         | 19/1095 [0

 12%|█▏        | 134/1095 [01:48<12:56,  1.24it/s]


 12%|█▏        | 135/1095 [01:49<12:58,  1.23it/s]


 12%|█▏        | 136/1095 [01:50<13:00,  1.23it/s]


 13%|█▎        | 137/1095 [01:51<12:56,  1.23it/s]


 13%|█▎        | 138/1095 [01:52<12:57,  1.23it/s]


 13%|█▎        | 139/1095 [01:52<12:55,  1.23it/s]


 13%|█▎        | 140/1095 [01:53<12:56,  1.23it/s]


 13%|█▎        | 141/1095 [01:54<12:57,  1.23it/s]


 13%|█▎        | 142/1095 [01:56<13:00,  1.22it/s]


 13%|█▎        | 143/1095 [01:56<12:58,  1.22it/s]


 13%|█▎        | 144/1095 [01:58<13:04,  1.21it/s]


 13%|█▎        | 145/1095 [01:59<13:00,  1.22it/s]


 13%|█▎        | 146/1095 [02:00<13:00,  1.22it/s]


 13%|█▎        | 147/1095 [02:00<12:55,  1.22it/s]


 14%|█▎        | 148/1095 [02:01<12:57,  1.22it/s]


 14%|█▎        | 149/1095 [02:02<12:56,  1.22it/s]


 14%|█▎        | 150/1095 [02:03<12:58,  1.21it/s]


 14%|█▍        | 151/1095 [02:05<13:01,  1.21it/s]


 14%|█▍        | 152/1095 [02:05<13:00,  1.21i

 36%|███▋      | 398/1095 [05:23<09:26,  1.23it/s]


 36%|███▋      | 399/1095 [05:23<09:25,  1.23it/s]


 37%|███▋      | 400/1095 [05:24<09:24,  1.23it/s]


 37%|███▋      | 401/1095 [05:25<09:23,  1.23it/s]


 37%|███▋      | 402/1095 [05:26<09:23,  1.23it/s]


 37%|███▋      | 403/1095 [05:27<09:22,  1.23it/s]


 37%|███▋      | 404/1095 [05:27<09:20,  1.23it/s]


 37%|███▋      | 405/1095 [05:29<09:22,  1.23it/s]


 37%|███▋      | 406/1095 [05:30<09:20,  1.23it/s]


 37%|███▋      | 407/1095 [05:30<09:18,  1.23it/s]


 37%|███▋      | 408/1095 [05:31<09:17,  1.23it/s]


 37%|███▋      | 409/1095 [05:32<09:17,  1.23it/s]


 37%|███▋      | 410/1095 [05:32<09:15,  1.23it/s]


 38%|███▊      | 411/1095 [05:32<09:14,  1.23it/s]


 38%|███▊      | 412/1095 [05:34<09:13,  1.23it/s]


 38%|███▊      | 413/1095 [05:34<09:11,  1.24it/s]


 38%|███▊      | 414/1095 [05:34<09:11,  1.24it/s]


 38%|███▊      | 415/1095 [05:36<09:10,  1.24it/s]


 38%|███▊      | 416/1095 [05:37<09:11,  1.23i

 60%|██████    | 662/1095 [08:36<05:37,  1.28it/s]


 61%|██████    | 663/1095 [08:36<05:36,  1.28it/s]


 61%|██████    | 664/1095 [08:37<05:36,  1.28it/s]


 61%|██████    | 665/1095 [08:38<05:35,  1.28it/s]


 61%|██████    | 666/1095 [08:39<05:34,  1.28it/s]


 61%|██████    | 667/1095 [08:41<05:34,  1.28it/s]


 61%|██████    | 668/1095 [08:42<05:33,  1.28it/s]


 61%|██████    | 669/1095 [08:42<05:32,  1.28it/s]


 61%|██████    | 670/1095 [08:43<05:31,  1.28it/s]


 61%|██████▏   | 671/1095 [08:43<05:30,  1.28it/s]


 61%|██████▏   | 672/1095 [08:44<05:30,  1.28it/s]


 61%|██████▏   | 673/1095 [08:46<05:30,  1.28it/s]


 62%|██████▏   | 674/1095 [08:46<05:29,  1.28it/s]


 62%|██████▏   | 675/1095 [08:48<05:28,  1.28it/s]


 62%|██████▏   | 676/1095 [08:48<05:27,  1.28it/s]


 62%|██████▏   | 677/1095 [08:49<05:26,  1.28it/s]


 62%|██████▏   | 678/1095 [08:50<05:25,  1.28it/s]


 62%|██████▏   | 679/1095 [08:50<05:24,  1.28it/s]


 62%|██████▏   | 680/1095 [08:51<05:24,  1.28i

 85%|████████▍ | 926/1095 [12:02<02:11,  1.28it/s]


 85%|████████▍ | 927/1095 [12:03<02:11,  1.28it/s]


 85%|████████▍ | 928/1095 [12:03<02:10,  1.28it/s]


 85%|████████▍ | 929/1095 [12:05<02:09,  1.28it/s]


 85%|████████▍ | 930/1095 [12:07<02:09,  1.28it/s]


 85%|████████▌ | 931/1095 [12:07<02:08,  1.28it/s]


 85%|████████▌ | 932/1095 [12:10<02:07,  1.28it/s]


 85%|████████▌ | 933/1095 [12:10<02:06,  1.28it/s]


 85%|████████▌ | 934/1095 [12:11<02:06,  1.28it/s]


 85%|████████▌ | 935/1095 [12:12<02:05,  1.28it/s]


 85%|████████▌ | 936/1095 [12:12<02:04,  1.28it/s]


 86%|████████▌ | 937/1095 [12:12<02:03,  1.28it/s]


 86%|████████▌ | 938/1095 [12:13<02:02,  1.28it/s]


 86%|████████▌ | 939/1095 [12:13<02:01,  1.28it/s]


 86%|████████▌ | 940/1095 [12:15<02:01,  1.28it/s]


 86%|████████▌ | 941/1095 [12:15<02:00,  1.28it/s]


 86%|████████▌ | 942/1095 [12:16<01:59,  1.28it/s]


 86%|████████▌ | 943/1095 [12:18<01:58,  1.28it/s]


 86%|████████▌ | 944/1095 [12:19<01:58,  1.28i

,t_index,tf
101,102,0.000137
115,116,0.000152
129,130,0.000119
130,131,0.000151
162,163,0.000044
224,225,0.000043
225,226,0.000117
242,243,0.000063
260,261,0.000203
462,463,0.000021


In [260]:
def cosine(DOCx, DOCy):
    '''
    input: doc1 path name(str) , doc2 path name(str)
    ouput: two doc's cosine similarity
    '''
    dfx = pd.read_csv(DOCx,sep=' ',names=['tindex','tfidf'],header=None)
    dfy = pd.read_csv(DOCy,sep=' ',names=['tindex','tfidf'],header=None)
    dfx = dfx.drop(0)
    dfy = dfy.drop(0)
    dfxy = pd.merge(dfx,dfy,on='tindex',how='outer')
    dfxy.fillna(0,inplace=True)
    up = sum(dfxy.tfidf_x * dfxy.tfidf_y)
    down = np.sqrt(sum(np.square(dfxy.tfidf_x)))* np.sqrt(sum(np.square(dfxy.tfidf_y)))
    result = up / down
    return result

In [264]:
cos = cosine('output/tf-idf/1.txt','output/tf-idf/2.txt')
cos

0.25903264882120364

In [259]:
dfc = pd.read_csv('output/tf-idf/1.txt',sep=' ',names=['tindex','tfidf'],header=None)
dfc = dfc.drop(0)
dfc_= pd.read_csv('output/tf-idf/2.txt',sep=' ',names=['tindex','tfidf'],header=None)
dfc_ = dfc_.drop(0)

In [253]:
df_all = pd.merge(dfc,dfc_,on='tindex',how='outer')
df_all.fillna(0,inplace=True)
df_all

,tindex,tfidf_x,tfidf_y
0,44,0.000106,0.000000
1,58,0.000040,0.000000
2,145,0.000058,0.000000
3,225,0.000043,0.000000
4,243,0.000063,0.000063
5,412,0.000064,0.000000
6,413,0.000076,0.000000
7,463,0.000021,0.000021
8,645,0.000063,0.000000
9,651,0.000126,0.000000


In [258]:
up = sum(df_all.tfidf_x * df_all.tfidf_y)
down = np.sqrt(sum(np.square(df_all.tfidf_x)))* np.sqrt(sum(np.square(df_all.tfidf_y)))
result = up / down
result

0.25903264882120364

In [225]:
df_tf.tf = (df_tf.tf / 8912)* np.log10(1095 / df2.df)
# df_tf.loc[df_tf.tf]
kk = df_tf[df_tf.tf != 0]
len(kk)

2718

In [151]:
df.df.std() , df.df.mean() , df.df.max() , df.df.min() , df.df.quantile(0.25) , df.df.quantile(0.5) , df.df.quantile(0.75)

(69.8764854400815, 25.22431961185657, 1094, 1, 1.0, 3.0, 14.0)

In [180]:
df.df.quantile(0.999999999) #>950拿掉、<2拿掉

1093.9979951200512

In [187]:
df.loc[df['df']>950]

,t_index,term,df
12123,12124,u,1094


In [132]:
def multi(file):
    f = open('data/IRTM/'+file)
    texts = f.read()
    f.close()
    tokens_all = preprocess(texts)
    for term in token_list:
        if term in tokens_all:
            df.loc[df['term']==term,'df'] += 1 # build the dict
    return df
def handle_error(e):
    '''處理 child process 的錯誤，不然 code 寫錯時，不會回報任何錯誤'''
    traceback.print_exception(type(e), e, e.__traceback__)
manager = Manager()
error_messages = manager.Queue()
with Pool(processes = 35) as pool:
    for file in next(os.walk('data/IRTM/'))[2]:
        pool.apply_async(multi,args=(file,),error_callback=handle_error)
    pool.close()
    pool.join()
#         print(file)
#         pool.map(multi, 'data/IRTM/'+file)
df

Exception in thread Thread-52:
Traceback (most recent call last):
  File "/home/leoqaz12/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/leoqaz12/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/leoqaz12/anaconda3/lib/python3.6/multiprocessing/pool.py", line 479, in _handle_results
    cache[job]._set(i, obj)
  File "/home/leoqaz12/anaconda3/lib/python3.6/multiprocessing/pool.py", line 651, in _set
    self._error_callback(self._value)
  File "<ipython-input-132-0ff770d3b5a6>", line 12, in handle_error
    traceback.print_exception(type(e), e, e.__traceback__)
NameError: name 'traceback' is not defined

Process ForkPoolWorker-568:
Process ForkPoolWorker-573:
Traceback (most recent call last):
Process ForkPoolWorker-560:
Process ForkPoolWorker-561:
Process ForkPoolWorker-566:
Traceback (most recent call last):
Process ForkPoolWorker-553:
Process ForkPoolWorker-569:
Proces

AssertionError: 

  File "/home/leoqaz12/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/leoqaz12/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/leoqaz12/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/leoqaz12/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/leoqaz12/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/leoqaz12/anaconda3/lib/python3.6/multiprocessing/pool.py", line 125, in worker
    put((job, i, result))
  File "/home/leoqaz12/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/leoqaz12/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/leoqaz12/anaconda3/lib/python3.6/multiprocessing/process.py", 

  File "/home/leoqaz12/anaconda3/lib/python3.6/multiprocessing/pool.py", line 125, in worker
    put((job, i, result))
  File "/home/leoqaz12/anaconda3/lib/python3.6/multiprocessing/pool.py", line 125, in worker
    put((job, i, result))
  File "/home/leoqaz12/anaconda3/lib/python3.6/multiprocessing/pool.py", line 125, in worker
    put((job, i, result))
  File "/home/leoqaz12/anaconda3/lib/python3.6/multiprocessing/pool.py", line 125, in worker
    put((job, i, result))
  File "/home/leoqaz12/anaconda3/lib/python3.6/multiprocessing/pool.py", line 125, in worker
    put((job, i, result))
  File "/home/leoqaz12/anaconda3/lib/python3.6/multiprocessing/pool.py", line 125, in worker
    put((job, i, result))
  File "/home/leoqaz12/anaconda3/lib/python3.6/multiprocessing/pool.py", line 125, in worker
    put((job, i, result))
  File "/home/leoqaz12/anaconda3/lib/python3.6/multiprocessing/pool.py", line 125, in worker
    put((job, i, result))
  File "/home/leoqaz12/anaconda3/lib/python3.6/m

  File "/home/leoqaz12/anaconda3/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/leoqaz12/anaconda3/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/leoqaz12/anaconda3/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/leoqaz12/anaconda3/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/leoqaz12/anaconda3/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/leoqaz12/anaconda3/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/leoqaz12/anaconda3/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/leoqaz12/anac

In [136]:
kk = [file for file in next(os.walk('data/IRTM/'))[2]]
df.loc[df['term']==term,'df'] , term

(13190    0
 Name: df, dtype: int64, 'zvezda')

In [106]:
df.loc[df['term']=='abc','df'] += 1
df

,t_index,term,df
0,1,abahd,0
1,2,abandon,0
2,3,abat,0
3,4,abc,4
4,5,abc.com,0
5,6,abcnews.com,0
6,7,abdallah,0
7,8,abdel,0
8,9,abdomin,0
9,10,abduct,0


In [92]:
tokens_all

'justin hanov cnn headlin sport venu william advanc australian open semifin tough-fought three-set victori south african amanda coetzer william pull decid set serena william point away play older sister venu semi top-seed martina hingi ralli spoil all-william affair win men sebastien grosjean advanc semi outclass spain carlo moya straight set '

In [79]:
# pickle.dump(token_list[:215],open('data/stop_list2.pkl','wb')) # 放進去stopwords
token_list[-3]

'zurich'